# Multithreaded Sharding

## Debugging the script

Open

In [1]:
!cat shard_parallel.py

import h5py
from mpi4py import MPI
import numpy as np
import os.path as osp

from kosmoss import CONFIG, PROCESSED_DATA_PATH

def main() -> None:

    timestep = CONFIG['timestep']
    h5_path = osp.join(PROCESSED_DATA_PATH, f'features-{timestep}.h5')

    rank = MPI.COMM_WORLD.rank
    print(f'worker of rank {rank} started.')

    for subidx in np.arange(53):
        
        print(f'processing slice {subidx} for rank {rank}.')
        start = rank * 53 * 2 ** 4 + subidx * 4800
        end = start + 4800

        with h5py.File(h5_path, 'r') as feats:

            sharded_path = osp.join(
                PROCESSED_DATA_PATH, 
                f'feats-{timestep}.{rank}.{subidx}.h5')
            with h5py.File(sharded_path, 'w') as sharded:

                sharded.create_dataset("/x", data=feats['/x'][start:end])
                sharded.create_dataset("/y", data=feats['/y'][start:end])
                sharded.create_dataset("/edge", data=feats['/edge'][start:end])
                
    p

## Executing the script

For some reason, `mpiexec` doesn't fancy being called from within a notebook?
You'll have to execute the follow command in a terminal manually from within the script directory.

Run `htop` on one terminal and run the following block in another one:
<code>
mpiexec -n 16 python shard_parallel.py
</code>